In [1]:
#svm
import menpo.io as mio
path_to_svm_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'

def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

training_images = mio.import_images(path_to_svm_database, verbose=True)
training_images = training_images.map(process)

Found 1186 assets, index the returned LazyList to import.


In [29]:
import os
class ChangeVector:
    def __init__(self, facs = [], landmarkChange = []):
        self.landmarkChange = landmarkChange
        self.facs = facs

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'
count = 0;
svm_training_data = []
while(count < 1186):
    file_path = str(training_images[count].path).split("\\")
    
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    facs = []
    for line in fi: # read rest of lines
        for x in line.split():
            facs.append(int(float(x)))
    #print(array)
    fi.close()
    
    landmarkChange = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmarkChange.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
    svm_training_data.append(ChangeVector(facs, landmarkChange))   
    count = count + 2

In [12]:
print(svm_training_data[400].facs)

[12.0, 3.0]


In [37]:
x_training = []
y_label = []
models = []
facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if(int(facs_code) not in facs):
            facs.append(facs_code)

#create model for each action unit in facs[]
for au in facs:
    #create label array
    for data in svm_training_data:
        if(au in data.facs):
            y_label.append(1)
        else:
            y_label.append(0)
        #create training data: 1x136 array
        tmp = []
        for vector in data.landmarkChange:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
    

[-2.6980930928635729, -0.31857955010475081, -2.4350712086726105, -0.32509667104587692, -2.4298610239616778, -0.35955207872170547, -2.1811436683827026, -0.28568767922696026, -1.9174666553812187, -0.20775041201777356, -2.7972784737445693, 1.1049142994780965, -2.9224991625867744, 1.3277957830916023, -1.23534760148749, 0.57934014892410346, -1.225183463165763, 0.82741473408228217, -3.3194317944334557, 0.1814099208488642, -4.8558478954794566, -0.36411945838878523, -5.2758224234044633, -0.82203783382576034, -4.2510266855160239, 0.049837009428216561, -2.9651567587876286, 3.4990404561355035, -2.9068832215120324, 2.0883269243263953, -2.7085528331971034, 1.7785721806339154, -2.7003767639251492, 1.778681892849761, -4.6190237758178228, 0.11972084774628655, -1.9490443023060706, 1.407556580458774, -0.11077651536596278, 1.3682180209973325, 1.0794218391341062, 0.96527008383105795, 4.3776095950579901, -1.1732056002104656, 8.5521781483821044, -1.199075533661528, 2.846780143311662, -2.6060924446163085, 0.